In [20]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from pprint import pprint
from config import gkey
gmaps.configure(gkey)

In [21]:
# Import previous data and create dataframe
weather_df = pd.read_csv("../weatherpy/output/weatherpy.csv", encoding="utf-8")
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,0,busselton,AU,-33.6500,115.3333,1619221263,93,75,61.00,4.00
1,1,punta arenas,PH,8.9897,125.3400,1619221771,99,72,83.19,1.54
2,2,rikitea,PF,-23.1203,-134.9692,1619221615,100,68,78.96,15.99
3,3,albany,US,42.6001,-73.9662,1619221771,20,33,55.40,18.41
4,4,kandi,BJ,11.1342,2.9386,1619221772,80,46,85.82,6.13
...,...,...,...,...,...,...,...,...,...,...
557,557,asosa,ET,10.0667,34.5333,1619221758,53,67,66.52,4.79
558,558,vitina,BA,43.2375,17.4839,1619221626,20,93,50.00,1.14
559,559,bagdarin,RU,54.4333,113.6000,1619221406,99,41,31.30,3.02
560,560,gogrial,SS,8.5332,28.1004,1619221759,37,49,84.16,4.76


In [22]:
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()
location = weather_df[["Latitude", "Longitude"]]

In [23]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
ideal_weather_df = weather_df.loc[(weather_df["Max Temperature"] > 70) & (weather_df["Max Temperature"] < 80) & (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] < 10), :]
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df['index']
ideal_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,28,riyadh,SA,24.6877,46.7219,1619221277,0,11,78.80,4.61
1,250,atar,MR,20.5169,-13.0499,1619221672,0,13,79.68,5.53
2,296,belmonte,BR,-15.8631,-38.8828,1619221251,0,69,78.80,6.91
3,361,jesus carranza,PY,-27.0500,-55.7833,1619221699,0,65,77.00,3.44
4,473,sakakah,SA,29.9697,40.2064,1619221731,0,13,77.00,9.22


In [25]:
hotels = []

for i in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[i]["Latitude"]
    lng = ideal_weather_df.loc[i]["Longitude"]

    parameter = {"location": f"{lat},{lng}", "radius": 5000, "types" : "hotel", "key": gkey}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=parameter)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
ideal_weather_df["Hotel Name"] = hotels
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name
0,28,riyadh,SA,24.6877,46.7219,1619221277,0,11,78.80,4.61,Riyadh
1,250,atar,MR,20.5169,-13.0499,1619221672,0,13,79.68,5.53,Atar
2,296,belmonte,BR,-15.8631,-38.8828,1619221251,0,69,78.80,6.91,Belmonte
3,361,jesus carranza,PY,-27.0500,-55.7833,1619221699,0,65,77.00,3.44,Jesús de Tavarangue
4,473,sakakah,SA,29.9697,40.2064,1619221731,0,13,77.00,9.22,Sakaka


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [28]:
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))